# GRPO Fine-Tuning for Financial Reasoning

This notebook implements a **training pipeline** for financial reasoning using GRPO (Group Relative Policy Optimization).

## Training Pipeline

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  1. Load Base   │ -> │ 2. Bootstrap    │ -> │ 3. SFT Phase    │ -> │ 4. GRPO Phase   │
│  Qwen2.5-7B     │    │ Data (Gemini)   │    │ (OPTIONAL)      │    │ (RL Fine-tune)  │
└─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
```

## Key Features

1. **Model**: Qwen2.5-7B-Instruct (good math/reasoning priors)
2. **Higher LoRA Rank**: r=64 for complex reasoning tasks
3. **Bootstrapping**: Gemini generates synthetic reasoning traces (optional)
4. **SFT Phase**: SKIPPED by default - base model already has ~80% format compliance
5. **GRPO Phase**: Reinforcement learning to improve accuracy

## Verbose Evaluation

Evaluation outputs show **full model responses** (no truncation) including:
- Complete question
- Ground truth answer
- Full model response with thinking and answer
- Extracted thinking and answer
- Accuracy and format compliance per question

## Configuration Options

| Option | Default | Description |
|--------|---------|-------------|
| `cleanup_old_checkpoints` | `False` | Delete old SFT/GRPO checkpoints before starting |
| `cleanup_bootstrap_data` | `False` | Regenerate bootstrap data from scratch |
| `skip_sft` | `True` | Skip SFT phase (recommended - base model has good format compliance) |

## Requirements

- NVIDIA GPU with CUDA
- `.env` file with `HF_TOKEN` and `GEMINI_API_KEY`


## Phase 1: Setup & Configuration


In [1]:
import os
import re
import json
import random
import shutil
import warnings
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import polars as pl
from datasets import load_dataset, Dataset
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Project paths
PROJECT_ROOT = Path().absolute().parent.parent
MODELS_DIR = PROJECT_ROOT / "models"
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = MODELS_DIR / "checkpoints" / "grpo-financial-v2"
BOOTSTRAP_FILE = DATA_DIR / "processed" / "bootstrap_data.json"

# Create directories
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(DATA_DIR / "processed").mkdir(parents=True, exist_ok=True)

# Load environment
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path if env_path.exists() else None)

# =============================================================================
# CONFIGURATION - Enhanced for better reasoning
# =============================================================================
CONFIG = {
    # ==========================================================================
    # CLEANUP - Set to True to delete old checkpoints before starting fresh
    # ==========================================================================
    "cleanup_old_checkpoints": False,  # True = delete old SFT/GRPO checkpoints
    "cleanup_bootstrap_data": False,   # True = regenerate bootstrap data from scratch
    
    # Model - Upgraded to 7B Instruct for better reasoning
    "model_name": "Qwen/Qwen2.5-7B-Instruct",
    "max_seq_length": 8192,
    "load_in_4bit": True,
    
    # LoRA - Higher rank for complex reasoning
    "lora_r": 64,
    "lora_alpha": 64,
    "lora_dropout": 0,
    
    # Bootstrap Data Generation (Gemini generates natural language reasoning traces)
    "bootstrap_samples": 500,
    "bootstrap_max_retries": 2,  # retries per sample if Gemini output fails validation
    
    # Gemini (REQUIRED for bootstrap data generation)
    "gemini_model": "gemini-2.5-pro",
    
    # SFT Phase - Skipped by default since base model already has good format compliance
    # Set to False if bootstrap data quality is high and you want to pre-train on it
    "skip_sft": False,  # True = skip SFT, go directly from base model to GRPO
    "sft_epochs": 1,
    "sft_learning_rate": 2e-5,
    "sft_batch_size": 1,
    "sft_gradient_accumulation": 4,
    
    # GRPO Phase
    "grpo_learning_rate": 5e-6,
    "grpo_max_steps": 200,
    "grpo_num_generations": 4,
    "grpo_max_completion_length": 2048,  # Longer for full reasoning traces
    
    # Evaluation
    "eval_samples": 20,
    "random_seed": 42,
    "max_new_tokens": 1024,

    # Eval printing/prompt controls
    # - prompt context is truncated to keep prompts stable
    # - printed context is shown to help debugging
    "eval_prompt_context_chars": 6000,
    "eval_print_context_chars": None,  # None = print full context

    # LoRA ordering
    "apply_lora_before_base_eval": False,  # True = base eval is "base+LoRA init" instead of pure base
}

# Validate API keys
HF_TOKEN = os.getenv("HF_TOKEN", "")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")

print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"\n📦 Model: {CONFIG['model_name']}")
print(f"   LoRA Rank: {CONFIG['lora_r']}")
print(f"   Max Seq Length: {CONFIG['max_seq_length']}")
print(f"\n📊 Bootstrap Samples: {CONFIG['bootstrap_samples']}")
print(f"   Gemini Model: {CONFIG['gemini_model']}")
sft_status = "SKIPPED" if CONFIG['skip_sft'] else f"{CONFIG['sft_epochs']} epoch(s)"
print(f"\n🎓 SFT: {sft_status}")
print(f"   GRPO: {CONFIG['grpo_max_steps']} steps")
print(f"\n🔑 HF Token: {'✓' if HF_TOKEN else '✗'}")
print(f"   Gemini API Key: {'✓' if GEMINI_API_KEY else '✗'}")
print("="*60)


CONFIGURATION

📦 Model: Qwen/Qwen2.5-7B-Instruct
   LoRA Rank: 64
   Max Seq Length: 8192

📊 Bootstrap Samples: 500
   Gemini Model: gemini-2.5-pro

🎓 SFT: 1 epoch(s)
   GRPO: 200 steps

🔑 HF Token: ✓
   Gemini API Key: ✓


In [2]:
# =============================================================================
# CLEANUP PHASE - Remove old checkpoints if requested
# =============================================================================
# This ensures a completely fresh start (the model is ALWAYS loaded fresh from
# HuggingFace regardless, but this cleans up old saved checkpoints from disk)

def cleanup_old_runs():
    """Clean up old checkpoints and optionally bootstrap data."""
    cleaned = []
    
    if CONFIG["cleanup_old_checkpoints"]:
        # Clean up SFT checkpoint
        sft_dir = OUTPUT_DIR / "sft-checkpoint"
        if sft_dir.exists():
            shutil.rmtree(sft_dir)
            cleaned.append(f"SFT checkpoint: {sft_dir}")
        
        # Clean up final model
        final_dir = OUTPUT_DIR / "final-model"
        if final_dir.exists():
            shutil.rmtree(final_dir)
            cleaned.append(f"Final model: {final_dir}")
        
        # Clean up any GRPO checkpoints
        for checkpoint_dir in OUTPUT_DIR.glob("checkpoint-*"):
            if checkpoint_dir.is_dir():
                shutil.rmtree(checkpoint_dir)
                cleaned.append(f"GRPO checkpoint: {checkpoint_dir}")
        
        # Clean up comparison results
        results_csv = OUTPUT_DIR / "comparison_results.csv"
        if results_csv.exists():
            results_csv.unlink()
            cleaned.append(f"Results CSV: {results_csv}")
    
    if CONFIG["cleanup_bootstrap_data"]:
        if BOOTSTRAP_FILE.exists():
            BOOTSTRAP_FILE.unlink()
            cleaned.append(f"Bootstrap data: {BOOTSTRAP_FILE}")
    
    return cleaned

# Run cleanup
cleaned_items = cleanup_old_runs()

if cleaned_items:
    print("🧹 CLEANUP COMPLETED:")
    for item in cleaned_items:
        print(f"  ✓ Deleted: {item}")
    print()
else:
    if CONFIG["cleanup_old_checkpoints"] or CONFIG["cleanup_bootstrap_data"]:
        print("🧹 Cleanup requested but nothing to clean up")
    else:
        print("ℹ️  Cleanup not requested (set cleanup_old_checkpoints=True to clean up)")

# Show what exists
print("\n📁 Current state of output directory:")
if OUTPUT_DIR.exists():
    for item in sorted(OUTPUT_DIR.iterdir()):
        size = sum(f.stat().st_size for f in item.rglob('*') if f.is_file()) / (1024*1024) if item.is_dir() else item.stat().st_size / (1024*1024)
        print(f"  {'📁' if item.is_dir() else '📄'} {item.name}: {size:.1f} MB")
else:
    print("  (empty)")

if BOOTSTRAP_FILE.exists():
    size = BOOTSTRAP_FILE.stat().st_size / (1024*1024)
    print(f"\n📄 Bootstrap data exists: {BOOTSTRAP_FILE.name} ({size:.1f} MB)")
else:
    print("\n📄 Bootstrap data: not generated yet")


ℹ️  Cleanup not requested (set cleanup_old_checkpoints=True to clean up)

📁 Current state of output directory:
  📄 comparison_results.csv: 0.0 MB
  📁 final-model: 631.2 MB
  📁 grpo-checkpoint: 1889.0 MB
  📁 sft-checkpoint: 1575.7 MB

📄 Bootstrap data: not generated yet


## Phase 2: Load & Prepare Dataset


In [3]:
# Load ConvFinQA dataset (structured HF dataset)
# Dataset card: https://huggingface.co/datasets/MehdiHosseiniMoghadam/ConvFinQA
print("Loading ConvFinQA (MehdiHosseiniMoghadam/ConvFinQA)...")
dataset_dict = load_dataset("MehdiHosseiniMoghadam/ConvFinQA")
train_ds = dataset_dict["train"]
test_ds = dataset_dict["test"]
print(f"✓ Loaded train: {len(train_ds):,} | test: {len(test_ds):,}")

# System prompt for financial reasoning (used for both SFT and GRPO)
# NOTE: No code generation - model should reason and answer directly
SYSTEM_PROMPT = """You are an expert financial analyst. Answer financial questions accurately using step-by-step reasoning.

Follow this exact format:

1. Think through the problem step-by-step in <think> tags
2. Provide the final answer in <answer> tags

Rules:
- Do NOT write any code.
- Put the final answer ONLY inside <answer>...</answer>.

Example:
<think>
We need percent change from $100M to $150M.
Percent change = (150 - 100) / 100 = 0.5 = 50%.
</think>

<answer>50%</answer>"""


def _render_table_markdown(table_rows: list[list[str]], max_rows: int = 18, max_cols: int = 10) -> str:
    """Render ConvFinQA 'table' field into compact markdown."""
    if not table_rows:
        return "(no table)"
    rows = table_rows[:max_rows]
    rows = [r[:max_cols] for r in rows]
    max_len = max(len(r) for r in rows)
    rows = [r + [""] * (max_len - len(r)) for r in rows]

    def esc(x):
        return str(x).replace("\n", " ").strip()

    header = rows[0]
    body = rows[1:]
    md = "| " + " | ".join(esc(x) for x in header) + " |\n"
    md += "| " + " | ".join(["---"] * len(header)) + " |\n"
    for r in body:
        md += "| " + " | ".join(esc(x) for x in r) + " |\n"
    return md


def build_context(row: dict, max_pre: int = 10, max_post: int = 8) -> str:
    """Build a clean context string from pre_text + table + post_text."""
    pre = row.get("pre_text") or []
    post = row.get("post_text") or []
    table = row.get("table") or []

    pre_txt = "\n".join(str(x) for x in pre[:max_pre])
    post_txt = "\n".join(str(x) for x in post[:max_post])
    table_md = _render_table_markdown(table)

    parts = []
    if pre_txt.strip():
        parts.append("PRE-TEXT:\n" + pre_txt.strip())
    parts.append("TABLE:\n" + table_md.strip())
    if post_txt.strip():
        parts.append("POST-TEXT:\n" + post_txt.strip())

    return "\n\n".join(parts).strip()


def format_sample(row: dict) -> dict:
    """Format a ConvFinQA sample for training/eval."""
    context = build_context(row)
    question = str(row["question"]).strip()
    answer = str(row["answer"]).strip()

    return {
        "id": row.get("id", ""),
        "filename": row.get("filename", ""),
        "context": context,
        "question": question,
        "answer": answer,
        "steps": row.get("steps", []),
        "prompt": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {question}"},
        ],
    }

# Process all samples
print("Formatting samples...")
train_rows = train_ds.to_list()
test_rows = test_ds.to_list()
train_df = pd.DataFrame([format_sample(r) for r in train_rows])
test_df = pd.DataFrame([format_sample(r) for r in test_rows])
print(f"✓ Processed train: {len(train_df):,} | test: {len(test_df):,}")

# Filter out invalid QA rows (missing/empty question or answer)
def _is_valid_qa_df(df: pd.DataFrame) -> pd.Series:
    q = df["question"].astype(str).fillna("").str.strip()
    a = df["answer"].astype(str).fillna("").str.strip()
    # Treat literal 'nan' as invalid (can happen after astype(str))
    q = q.mask(q.str.lower() == "nan", "")
    a = a.mask(a.str.lower() == "nan", "")
    return (q != "") & (a != "")

train_mask = _is_valid_qa_df(train_df)
test_mask = _is_valid_qa_df(test_df)

train_dropped = int((~train_mask).sum())
test_dropped = int((~test_mask).sum())

train_df = train_df[train_mask].reset_index(drop=True)
test_df = test_df[test_mask].reset_index(drop=True)

print(f"✓ Filtered invalid QA rows")
print(f"  Train: dropped {train_dropped:,}, kept {len(train_df):,}")
print(f"  Test:  dropped {test_dropped:,}, kept {len(test_df):,}")

if len(test_df) == 0:
    raise ValueError("After filtering, test split has 0 valid QA rows")

# Quick sanity check
print("\nSample (train):")
print(f"  ID: {train_df.iloc[0].get('id','')}")
print(f"  Q: {train_df.iloc[0]['question'][:120]}...")
print(f"  A: {train_df.iloc[0]['answer']}")
print(f"  Context (first 200 chars): {train_df.iloc[0]['context'][:200]}...")


Loading ConvFinQA (MehdiHosseiniMoghadam/ConvFinQA)...


Repo card metadata block was not found. Setting CardData to empty.


✓ Loaded train: 3,037 | test: 200
Formatting samples...
✓ Processed train: 3,037 | test: 200
✓ Filtered invalid QA rows
  Train: dropped 947, kept 2,090
  Test:  dropped 47, kept 153

Sample (train):
  ID: Single_JKHY/2009/page_28.pdf-3
  Q: what was the percentage change in the net cash from operating activities from 2008 to 2009...
  A: 14.1%
  Context (first 200 chars): PRE-TEXT:
26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 .
all revenue components within the segmen...


In [4]:
# Split dataset using ConvFinQA official splits
# - Evaluation: sample from official test split
# - Bootstrap/GRPO: sampled from train split
random.seed(CONFIG["random_seed"])

# Evaluation comes from the official test split
if len(test_df) == 0:
    raise ValueError("ConvFinQA test split is empty - cannot evaluate")

eval_n = min(CONFIG["eval_samples"], len(test_df))
eval_df = test_df.sample(n=eval_n, random_state=CONFIG["random_seed"]).reset_index(drop=True)

# Bootstrap + GRPO come from the train split
train_indices = list(range(len(train_df)))

bootstrap_n = min(CONFIG["bootstrap_samples"], len(train_indices))
bootstrap_indices = random.sample(train_indices, bootstrap_n)
grpo_indices = [i for i in train_indices if i not in bootstrap_indices]

bootstrap_df = train_df.iloc[bootstrap_indices].reset_index(drop=True)
grpo_df = train_df.iloc[grpo_indices].reset_index(drop=True)

print(f"✓ Dataset splits:")
print(f"   Evaluation (test split): {len(eval_df)} samples")
print(f"   Bootstrap (train split): {len(bootstrap_df)} samples")
print(f"   GRPO training (train split): {len(grpo_df)} samples")

# Show sample
print(f"\nSample question (eval):")
print(f"  ID: {eval_df.iloc[0].get('id','')}")
print(f"  Q: {eval_df.iloc[0]['question'][:120]}...")
print(f"  A: {eval_df.iloc[0]['answer']}")

print(f"\nSample question (train):")
print(f"  ID: {train_df.iloc[0].get('id','')}")
print(f"  Q: {train_df.iloc[0]['question'][:120]}...")
print(f"  A: {train_df.iloc[0]['answer']}")


✓ Dataset splits:
   Evaluation (test split): 20 samples
   Bootstrap (train split): 500 samples
   GRPO training (train split): 1590 samples

Sample question (eval):
  ID: Single_KIM/2008/page_126.pdf-2
  Q: what is the growth rate in sublease revenues from 2007 to 2008?...
  A: -7.8%

Sample question (train):
  ID: Single_JKHY/2009/page_28.pdf-3
  Q: what was the percentage change in the net cash from operating activities from 2008 to 2009...
  A: 14.1%


## Phase 3: Generate Synthetic Bootstrap Data with Gemini

This phase uses Gemini to generate **natural language reasoning traces** for training. 

**Hybrid Approach:**
- Gemini generates natural language thinking (explains *why* each calculation is done)
- ConvFinQA's `steps` field guides the reasoning (shows what calculations to perform)
- Ground truth answer validates the output (must match exactly)

These traces teach the model:
1. How to use `<think>` tags for step-by-step **natural language** reasoning
2. How to reference values from the context (tables, text)
3. How to format final answers in `<answer>` tags


In [5]:
import time

# Gemini is REQUIRED for bootstrap data generation
try:
    from google import genai
except ImportError:
    genai = None

# Initialize Gemini client (required for bootstrap)
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY is required for bootstrap data generation. Set it in .env file.")
if genai is None:
    raise ImportError("google-genai package is required. Install with: pip install google-genai")

gemini_client = genai.Client(api_key=GEMINI_API_KEY)
print(f"✓ Gemini client initialized with model: {CONFIG['gemini_model']}")

# =============================================================================
# PROMPT TEMPLATE: Generate natural language reasoning guided by steps
# =============================================================================
# The prompt:
# 1. Provides the question and context
# 2. Shows the calculation steps (from ConvFinQA) as a GUIDE
# 3. Asks Gemini to write natural language reasoning that explains WHY
# 4. Requires the answer to match exactly
# =============================================================================

GEMINI_REASONING_PROMPT = """You are an expert financial analyst. Your task is to solve a financial question using NATURAL LANGUAGE reasoning.

**CONTEXT:**
{context}

**QUESTION:**
{question}

**CALCULATION GUIDE (for reference only - do NOT copy this format):**
{steps_description}

**EXPECTED ANSWER (your answer MUST match this exactly):** {answer}

Write your response in this EXACT format:

<think>
[Write a clear, natural language explanation of your reasoning. Explain:
- What values you're looking for in the context
- Why you need to perform each calculation
- Show each step with actual numbers from the context
- End with the final calculation that produces the answer]
</think>

<answer>{answer}</answer>

CRITICAL RULES:
1. Write in natural language - DO NOT use function notation like "add(x, y)" or "divide(#0, #1)"
2. Reference actual values from the context (e.g., "Revenue in 2023 was $100M")
3. Explain WHY each calculation is needed, not just what it is
4. Your <answer> MUST be exactly: {answer}
5. Do NOT write any code
6. Keep reasoning concise but complete (3-6 sentences typically)

Example of GOOD reasoning:
"Looking at the table, total revenue for 2023 was $100M and for 2024 was $200M. To find the growth rate, I need to calculate the change divided by the original: ($200M - $100M) / $100M = $100M / $100M = 100% growth."

Example of BAD reasoning (DO NOT DO THIS):
"Step 1: add(100, 200) = 300. Step 2: divide(#0, 100) = 3."
"""


def format_steps_for_prompt(steps: list) -> str:
    """Convert ConvFinQA steps to a human-readable description for the prompt."""
    if not steps or not isinstance(steps, list):
        return "No calculation steps provided."
    
    lines = []
    for i, step in enumerate(steps):
        if not isinstance(step, dict):
            continue
        op = str(step.get("op", "")).lower()
        a1 = str(step.get("arg1", "")).strip()
        a2 = str(step.get("arg2", "")).strip()
        res = str(step.get("res", "")).strip()
        
        # Convert operation to readable form
        if "minus" in op or "subtract" in op:
            desc = f"Subtract: {a1} - {a2} = {res}"
        elif "add" in op or "sum" in op:
            desc = f"Add: {a1} + {a2} = {res}"
        elif "divide" in op:
            desc = f"Divide: {a1} / {a2} = {res}"
        elif "multiply" in op or "times" in op:
            desc = f"Multiply: {a1} × {a2} = {res}"
        else:
            desc = f"Operation {op}: {a1}, {a2} → {res}"
        
        lines.append(f"  Step {i+1}: {desc}")
    
    return "\n".join(lines) if lines else "No valid steps."


print("✓ Gemini prompt template defined")


✓ Gemini client initialized with model: gemini-2.5-pro
✓ Gemini prompt template defined


In [6]:
def _extract_answer_from_trace(trace: str) -> str:
    """Extract the answer from a trace that contains <answer>...</answer> tags."""
    if not trace:
        return ""
    m = re.search(r"<answer>(.*?)(?:</answer>|$)", trace, re.DOTALL | re.IGNORECASE)
    return m.group(1).strip() if m else ""


def generate_gemini_trace(context: str, question: str, answer: str, steps: list) -> tuple[str, str | None]:
    """
    Generate a natural language reasoning trace using Gemini.
    
    The steps from ConvFinQA are provided as a GUIDE to help Gemini understand
    what calculations are needed, but Gemini writes natural language reasoning.
    
    Returns: (trace, error_message) - trace is None if failed
    """
    if gemini_client is None:
        return None, "Gemini client not initialized"
    
    # Format steps as a readable guide
    steps_description = format_steps_for_prompt(steps)
    
    # Build the prompt
    prompt = GEMINI_REASONING_PROMPT.format(
        context=str(context).strip()[:6000],  # Limit context length
        question=str(question).strip(),
        steps_description=steps_description,
        answer=str(answer).strip(),
    )
    
    try:
        resp = gemini_client.models.generate_content(
            model=CONFIG["gemini_model"],
            contents=prompt,
        )
        text = getattr(resp, "text", None) or ""
        if not text:
            return None, "Empty response from Gemini"
        return text, None
    except Exception as e:
        return None, str(e)


def validate_trace(trace: str, expected_answer: str | None = None) -> tuple[bool, list]:
    """Validate trace format and optionally enforce exact answer match."""
    if not trace:
        return False, ["Empty trace"]

    issues = []

    has_think_open = "<think>" in trace
    has_think_close = "</think>" in trace
    if not has_think_open:
        issues.append("Missing <think> opening tag")
    if not has_think_close:
        issues.append("Missing </think> closing tag")

    has_answer_open = "<answer>" in trace
    has_answer_close = "</answer>" in trace
    if not has_answer_open:
        issues.append("Missing <answer> opening tag")
    if not has_answer_close:
        issues.append("Missing </answer> closing tag")

    if expected_answer is not None:
        got = _extract_answer_from_trace(trace).strip()
        exp = str(expected_answer).strip()
        if got != exp:
            issues.append(f"Answer mismatch: got '{got}' expected '{exp}'")

    return len(issues) == 0, issues


def check_existing_bootstrap_data(output_file: Path) -> list:
    """Check if valid bootstrap data already exists."""
    if not output_file.exists():
        print(f"📂 Bootstrap file does not exist: {output_file}")
        return None

    try:
        with open(output_file, "r") as f:
            data = json.load(f)

        if not isinstance(data, list):
            print("⚠ Bootstrap file exists but is not a list")
            return None
        if len(data) == 0:
            print("⚠ Bootstrap file exists but is empty")
            return None

        required_fields = ["context", "question", "answer", "steps", "reasoning_trace", "trace_source"]
        first = data[0]
        missing = [f for f in required_fields if f not in first]
        if missing:
            print(f"⚠ Bootstrap data missing fields: {missing}")
            return None

        print(f"✓ Found existing bootstrap data: {len(data)} samples")
        return data

    except json.JSONDecodeError as e:
        print(f"⚠ Bootstrap file exists but is invalid JSON: {e}")
        return None


def generate_bootstrap_data(bootstrap_df: pd.DataFrame, output_file: Path, verbose: bool = True) -> list:
    """
    Generate synthetic training data using Gemini to create natural language reasoning traces.
    
    The steps from ConvFinQA are used as a GUIDE for Gemini to understand the calculations,
    but Gemini writes natural, human-readable reasoning. The steps also serve as validation
    to ensure the answer is correct.
    """

    existing_data = check_existing_bootstrap_data(output_file)
    if existing_data is not None:
        print(f"  ℹ️  Skipping generation - using existing {len(existing_data)} samples")
        return existing_data

    max_retries = int(CONFIG.get("bootstrap_max_retries", 2))

    print(f"\n{'='*70}")
    print("🚀 GENERATING BOOTSTRAP DATA WITH GEMINI")
    print(f"{'='*70}")
    print(f"  Samples to process: {len(bootstrap_df)}")
    print(f"  Gemini model: {CONFIG['gemini_model']}")
    print(f"  Max retries per sample: {max_retries}")
    print(f"  Output file: {output_file}")
    print(f"{'='*70}\n")

    bootstrap_data = []
    failed = 0
    error_types = {}

    for i, (_, row) in enumerate(bootstrap_df.iterrows()):
        sample_num = i + 1
        q = str(row.get("question", "")).strip()
        a = str(row.get("answer", "")).strip()
        ctx = str(row.get("context", "")).strip()
        steps = row.get("steps", [])

        if not q or not a:
            failed += 1
            error_types["missing_q_or_a"] = error_types.get("missing_q_or_a", 0) + 1
            continue

        # Try generating with Gemini (with retries)
        trace = None
        last_error = None
        for attempt in range(max_retries + 1):
            generated_trace, err = generate_gemini_trace(ctx, q, a, steps)
            if err:
                last_error = err
                time.sleep(0.5)  # Brief pause between retries
                continue
            
            # Validate the generated trace
            ok, issues = validate_trace(generated_trace, expected_answer=a)
            if ok:
                trace = generated_trace
                break
            else:
                last_error = f"Validation failed: {'; '.join(issues)}"
                time.sleep(0.3)
        
        if trace is None:
            failed += 1
            error_key = str(last_error)[:60] if last_error else "unknown_error"
            error_types[error_key] = error_types.get(error_key, 0) + 1
            if verbose:
                print(f"\n❌ [{sample_num}/{len(bootstrap_df)}] FAILED: {last_error}")
            continue

        if verbose:
            print(f"\n{'─'*70}")
            print(f"✅ [{sample_num}/{len(bootstrap_df)}] SUCCESS")
            print(f"❓ Q: {q[:140]}")
            print(f"📊 A: {a}")
            print(f"🧾 Context (first 200 chars): {ctx[:200]}...")
            print(f"🧩 Steps: {steps}")
            print(f"\n📝 GEMINI TRACE:\n{trace}")

        bootstrap_data.append({
            "context": ctx,
            "question": q,
            "answer": a,
            "steps": steps,
            "reasoning_trace": trace,
            "trace_source": "gemini",
        })

        # Checkpoint every 50 samples
        if len(bootstrap_data) > 0 and len(bootstrap_data) % 50 == 0:
            with open(output_file, "w") as f:
                json.dump(bootstrap_data, f, indent=2)
            print(f"\n💾 Checkpoint saved: {len(bootstrap_data)} samples")

        # Brief pause to avoid rate limits
        time.sleep(0.2)

    # Final save
    with open(output_file, "w") as f:
        json.dump(bootstrap_data, f, indent=2)

    print(f"\n{'='*70}")
    print("✅ BOOTSTRAP DATA GENERATION COMPLETE")
    print(f"{'='*70}")
    print(f"  ✓ Successful: {len(bootstrap_data)}")
    print(f"  ✗ Failed: {failed}")
    print(f"  📁 Saved to: {output_file}")
    if error_types:
        print(f"\n  Error breakdown:")
        for err, count in sorted(error_types.items(), key=lambda x: -x[1]):
            print(f"    - {err}: {count}")
    print(f"{'='*70}")

    return bootstrap_data

print("✓ Bootstrap generation functions defined (Gemini-only natural language traces)")


✓ Bootstrap generation functions defined (Gemini-only natural language traces)


In [7]:
# =============================================================================
# TEST: Gemini natural language trace generation
# =============================================================================
print("="*70)
print("🧪 TESTING GEMINI TRACE GENERATION WITH ONE SAMPLE")
print("="*70)

test_sample = bootstrap_df.iloc[0]
q = str(test_sample.get('question','')).strip()
a = str(test_sample.get('answer','')).strip()
ctx = str(test_sample.get('context','')).strip()
steps = test_sample.get('steps', [])

print(f"\n📋 TEST INPUT:")
print(f"{'─'*70}")
print(f"Question: {q}")
print(f"Ground Truth Answer: {a}")
print(f"Steps (used as guide): {steps}")
print(f"\nContext (first 500 chars):\n{ctx[:500]}...")

# Generate trace with Gemini
print(f"\n✨ Generating natural language trace with Gemini ({CONFIG['gemini_model']})...")
trace, err = generate_gemini_trace(ctx, q, a, steps)

if err:
    print(f"\n❌ GENERATION FAILED: {err}")
else:
    ok, issues = validate_trace(trace, expected_answer=a)
    status = "✅ VALID" if ok else "⚠️ INVALID"
    print(f"\n{status} (answer matches: {ok})")
    if issues:
        print(f"Issues: {issues}")
    print(f"\n{'─'*70}")
    print("GENERATED TRACE:")
    print(f"{'─'*70}")
    print(trace)
    print(f"{'─'*70}")
    
    # Show extracted answer for verification
    extracted = _extract_answer_from_trace(trace)
    print(f"\n📊 Extracted answer: '{extracted}'")
    print(f"📊 Expected answer:  '{a}'")
    print(f"📊 Match: {extracted == a}")


🧪 TESTING GEMINI TRACE GENERATION WITH ONE SAMPLE

📋 TEST INPUT:
──────────────────────────────────────────────────────────────────────
Question: what is the expected growth rate in operating leases from 2019 to 2020?
Ground Truth Answer: -9.1%
Steps (used as guide): [{'arg1': '1888', 'arg2': '2078', 'op': 'minus1-1', 'res': '-190'}, {'arg1': '#0', 'arg2': '2078', 'op': 'divide1-2', 'res': '-9.1%'}]

Context (first 500 chars):
PRE-TEXT:
note 11 .
commitments and contingencies commitments leases the company fffds corporate headquarters is located in danvers , massachusetts .
this facility encompasses most of the company fffds u.s .
operations , including research and development , manufacturing , sales and marketing and general and administrative departments .
in october 2017 , the acquired its corporate headquarters for approximately $ 16.5 million and terminated its existing lease arrangement ( see note 6 ) .
future ...

✨ Generating natural language trace with Gemini (gemini-2.5-pro)

In [8]:
# =============================================================================
# GENERATE BOOTSTRAP DATA (Full run)
# =============================================================================
# Gemini generates natural language reasoning traces guided by the ConvFinQA steps.
# The steps ensure mathematical correctness while Gemini provides human-readable reasoning.

bootstrap_data = generate_bootstrap_data(bootstrap_df, BOOTSTRAP_FILE, verbose=True)

# Summary
if bootstrap_data:
    print(f"\n✅ Bootstrap data ready: {len(bootstrap_data)} samples")

    # Show first sample
    print("\n" + "="*70)
    print("SAMPLE FROM BOOTSTRAP DATA")
    print("="*70)
    sample = bootstrap_data[0]
    print(f"Source: {sample.get('trace_source', 'gemini')}")
    print(f"Question: {sample['question']}")
    print(f"Answer: {sample['answer']}")
    print(f"Steps (guide): {sample.get('steps', [])}")
    print(f"\nReasoning Trace:\n{sample['reasoning_trace']}")
else:
    print("\n⚠ No bootstrap data available - SFT will be skipped")


📂 Bootstrap file does not exist: /workspace/data/processed/bootstrap_data.json

🚀 GENERATING BOOTSTRAP DATA WITH GEMINI
  Samples to process: 500
  Gemini model: gemini-2.5-pro
  Max retries per sample: 2
  Output file: /workspace/data/processed/bootstrap_data.json


──────────────────────────────────────────────────────────────────────
✅ [1/500] SUCCESS
❓ Q: what is the expected growth rate in operating leases from 2019 to 2020?
📊 A: -9.1%
🧾 Context (first 200 chars): PRE-TEXT:
note 11 .
commitments and contingencies commitments leases the company fffds corporate headquarters is located in danvers , massachusetts .
this facility encompasses most of the company fffd...
🧩 Steps: [{'arg1': '1888', 'arg2': '2078', 'op': 'minus1-1', 'res': '-190'}, {'arg1': '#0', 'arg2': '2078', 'op': 'divide1-2', 'res': '-9.1%'}]

📝 GEMINI TRACE:
<think>
To determine the expected growth rate in operating leases from 2019 to 2020, I need to find the values for those two years in the provided table. The tab

## Phase 4: Load Model & Configure LoRA


In [9]:
from unsloth import FastLanguageModel

print(f"Loading {CONFIG['model_name']}...")
print(f"  Max Seq Length: {CONFIG['max_seq_length']}")
print(f"  4-bit Quantization: {CONFIG['load_in_4bit']}")

model, tokenizer = FastLanguageModel.from_pretrained(
    CONFIG["model_name"],
    max_seq_length=CONFIG["max_seq_length"],
    load_in_4bit=CONFIG["load_in_4bit"],
    fast_inference=False,
    device_map="cuda:0",
)

print(f"\n✓ Model loaded: {type(model).__name__}")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading Qwen/Qwen2.5-7B-Instruct...
  Max Seq Length: 8192
  4-bit Quantization: True
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0a0+b558c986e8.nv25.11. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✓ Model loaded: Qwen2ForCausalLM


In [10]:
# LoRA helpers
# We evaluate the *true base model* first, then attach LoRA before SFT/GRPO.

def apply_lora(model):
    """Attach LoRA adapters (idempotent-ish: skips if already attached)."""
    # If already a PEFT model, skip
    if getattr(model, "peft_config", None):
        print("ℹ️  LoRA already attached - skipping")
        return model

    print("Configuring LoRA adapters...")
    print(f"  Rank: {CONFIG['lora_r']} (higher for complex reasoning)")
    print(f"  Alpha: {CONFIG['lora_alpha']}")

    model = FastLanguageModel.get_peft_model(
        model,
        r=CONFIG["lora_r"],
        lora_alpha=CONFIG["lora_alpha"],
        lora_dropout=CONFIG["lora_dropout"],
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
        use_gradient_checkpointing="unsloth",
        random_state=CONFIG["random_seed"],
    )

    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"\n✓ LoRA configured")
    print(f"  Trainable: {trainable:,} params ({100*trainable/total:.2f}%)")
    print(f"  Total: {total:,} params")
    return model


# Optionally attach LoRA before base evaluation (generally not recommended)
if CONFIG.get("apply_lora_before_base_eval", False):
    model = apply_lora(model)
else:
    print("ℹ️  LoRA will be attached AFTER base evaluation")


ℹ️  LoRA will be attached AFTER base evaluation


## Phase 5: Evaluate Base Model (Before Any Training)

First, we evaluate the base model to establish a baseline. This is BEFORE any SFT or GRPO training.


In [11]:
def generate_response_with_chat_template(model, tokenizer, messages: list, max_new_tokens: int = 1024) -> str:
    """Generate a response using the proper chat template format."""
    # Apply chat template for proper formatting
    prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=CONFIG["max_seq_length"] - max_new_tokens
    ).to("cuda")
    
    # Remember the input length in tokens
    input_length = inputs["input_ids"].shape[1]
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    
    # Extract ONLY the new tokens (not the prompt tokens)
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    
    return response


def extract_answer(response: str) -> str:
    """Extract answer from model response."""
    # Try exact tag match first
    match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Try partial tag match (opening tag only)
    match = re.search(r'<answer>(.*?)(?:</answer>|$)', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Fallback: look for "answer:" or similar patterns
    match = re.search(r'(?:answer|result|final answer)[:\s]+([^\n]+)', response, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    
    # Last resort: find numbers
    numbers = re.findall(r'-?\d+\.?\d*%?', response)
    return numbers[-1] if numbers else "N/A"


def extract_thinking(response: str) -> str:
    """Extract thinking from model response."""
    # Try exact tag match
    match = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # Try partial tag match (opening tag only, content until answer or end)
    match = re.search(r'<think>(.*?)(?:</think>|<answer>|$)', response, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # No tags - return everything before the answer as "thinking"
    match = re.search(r'<answer>', response)
    if match:
        return response[:match.start()].strip()
    
    return ""


def check_format_compliance(response: str) -> dict:
    """
    Check format compliance with lenient matching.
    Returns dict with detailed format info.
    """
    result = {
        "has_think_tags": '<think>' in response and '</think>' in response,
        "has_answer_tags": '<answer>' in response and '</answer>' in response,
        "has_think_open": '<think>' in response,
        "has_answer_open": '<answer>' in response,
        "has_reasoning_content": False,
        "has_answer_content": False,
    }
    
    # Check for actual reasoning content (even without perfect tags)
    # Look for thinking patterns: calculations, step-by-step, numbers in context
    thinking = extract_thinking(response)
    if thinking and len(thinking) > 20:
        result["has_reasoning_content"] = True
    
    # Check for answer content
    answer = extract_answer(response)
    if answer and answer != "N/A":
        result["has_answer_content"] = True
    
    # Format is OK if we have both reasoning and answer content
    # OR if we have proper tags for both
    result["format_ok"] = (
        (result["has_think_tags"] and result["has_answer_tags"]) or
        (result["has_reasoning_content"] and result["has_answer_content"])
    )
    
    return result


def check_accuracy(predicted: str, ground_truth: str) -> bool:
    """Check if prediction matches ground truth (with tolerance)."""
    pred = str(predicted).strip().lower()
    truth = str(ground_truth).strip().lower()
    
    if pred == truth:
        return True
    
    try:
        pred_nums = re.findall(r'-?\d+\.?\d*', pred)
        truth_nums = re.findall(r'-?\d+\.?\d*', truth)
        if pred_nums and truth_nums:
            pred_val = float(pred_nums[-1])
            truth_val = float(truth_nums[-1])
            if abs(truth_val) < 1e-10:
                return abs(pred_val) < 0.05
            return abs(pred_val - truth_val) / abs(truth_val) < 0.05
    except:
        pass
    return False


def evaluate_model(model, tokenizer, eval_df, label: str = "Model", verbose: bool = True):
    """Evaluate model on evaluation samples with verbose output including context."""
    print(f"\n{'='*80}")
    print(f"📊 Evaluating: {label}")
    print(f"{'='*80}")

    results = []
    correct = 0
    skipped = 0

    def _safe_str(x) -> str:
        if x is None:
            return ""
        s = str(x)
        # pandas can show NaN as 'nan'
        return "" if s.strip().lower() == "nan" else s

    for idx, row in eval_df.iterrows():
        # Pull fields robustly
        q = _safe_str(row.get("question", "")).strip()
        gt = _safe_str(row.get("answer", "")).strip()
        ctx = _safe_str(row.get("context", "")).strip()
        ex_id = _safe_str(row.get("id", "")).strip()
        filename = _safe_str(row.get("filename", "")).strip()

        # Skip invalid QA rows entirely (do not train/evaluate on them)
        if not q or not gt:
            # This should be rare now that we filter train_df/test_df, but keep as a safety net
            skipped += 1
            continue

        if not ctx:
            ctx = "(missing context)"

        # Context in the prompt (truncate for stability)
        ctx_for_prompt = ctx
        limit = CONFIG.get("eval_prompt_context_chars", 6000)
        if isinstance(limit, int) and limit > 0:
            ctx_for_prompt = ctx_for_prompt[:limit]

        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{ctx_for_prompt}\n\nQuestion: {q}"},
        ]

        response = generate_response_with_chat_template(model, tokenizer, messages, CONFIG["max_new_tokens"])
        predicted = extract_answer(response)

        is_correct = check_accuracy(predicted, gt)
        if is_correct:
            correct += 1

        format_info = check_format_compliance(response)

        results.append({
            "id": ex_id,
            "filename": filename,
            "question": q,
            "ground_truth": gt,
            "predicted": predicted,
            "correct": is_correct,
            "format_ok": format_info["format_ok"],
            "has_think_tags": format_info["has_think_tags"],
            "has_answer_tags": format_info["has_answer_tags"],
            "has_reasoning_content": format_info["has_reasoning_content"],
            "has_answer_content": format_info["has_answer_content"],
            "full_response": response,
        })

        if verbose:
            print(f"\n{'─'*80}")
            header = f"[{idx+1}/{len(eval_df)}]"
            if ex_id:
                header += f" ID: {ex_id}"
            if filename:
                header += f" | File: {filename}"
            print(header)

            # Print context (optionally truncated)
            print(f"\n🧾 CONTEXT:")
            ctx_print_limit = CONFIG.get("eval_print_context_chars", None)
            if isinstance(ctx_print_limit, int) and ctx_print_limit > 0:
                print(ctx[:ctx_print_limit])
                if len(ctx) > ctx_print_limit:
                    print(f"\n... (context truncated for display: {ctx_print_limit} chars) ...")
            else:
                print(ctx)

            print(f"\n❓ QUESTION:\n{q}")
            print(f"\n✅ GROUND TRUTH:\n{gt}")

            print(f"\n🤖 MODEL RESPONSE (full, no truncation):")
            print(f"{'─'*40}")
            print(response)
            print(f"{'─'*40}")

            print(f"\n📝 EXTRACTED ANSWER: {predicted}")

            status = "✓ CORRECT" if is_correct else "✗ WRONG"
            format_ok = "✓" if format_info["format_ok"] else "✗"
            print(f"\n📊 RESULT: {status}")
            print(f"   Format: {format_ok} (tags: think={format_info['has_think_tags']}, answer={format_info['has_answer_tags']})")
            print(f"   Content: reasoning={format_info['has_reasoning_content']}, answer={format_info['has_answer_content']}")
        else:
            status = "✓" if is_correct else "✗"
            format_ok = "✓" if format_info["format_ok"] else "✗"
            gt_short = gt.replace("\n", " ")[:25]
            pred_short = str(predicted).replace("\n", " ")[:25]
            print(f"[{idx+1}/{len(eval_df)}] {status} GT: {gt_short} | Pred: {pred_short} | Format:{format_ok}")

    if len(results) == 0:
        raise ValueError("No valid evaluation rows (question+answer) were available after filtering/skipping")

    accuracy = 100 * correct / len(results)
    format_rate = 100 * sum(r["format_ok"] for r in results) / len(results)

    print(f"\n{'='*80}")
    print(f"📊 {label} - FINAL RESULTS:")
    print(f"{'='*80}")
    denom = len(results)
    print(f"  Accuracy: {accuracy:.1f}% ({correct}/{denom})")
    print(f"  Format compliance: {format_rate:.1f}%")
    if skipped:
        print(f"  Skipped invalid QA rows during eval loop: {skipped}")
    print(f"  (Format = proper tags OR reasoning+answer content detected)")
    print(f"{'='*80}")

    return results, accuracy, format_rate

print("✓ Evaluation functions defined (with lenient format checking)")


✓ Evaluation functions defined (with lenient format checking)


In [12]:
# Evaluate BASE model (before any training)
# NOTE: This is the *true base model* unless apply_lora_before_base_eval=True
print("📊 Evaluating BASE model (no training yet)...")
base_label = "BASE Model (Before Any Training)"
if CONFIG.get("apply_lora_before_base_eval", False):
    base_label = "BASE+LoRA-init (Before Any Training)"

base_results, base_accuracy, base_format_rate = evaluate_model(
    model, tokenizer, eval_df, base_label
)

📊 Evaluating BASE model (no training yet)...

📊 Evaluating: BASE Model (Before Any Training)

────────────────────────────────────────────────────────────────────────────────
[1/20] ID: Single_KIM/2008/page_126.pdf-2 | File: KIM/2008/page_126.pdf

🧾 CONTEXT:
PRE-TEXT:
kimco realty corporation and subsidiaries notes to consolidated financial statements , continued investment in retail store leases the company has interests in various retail store leases relating to the anchor store premises in neighborhood and community shopping centers .
these premises have been sublet to retailers who lease the stores pursuant to net lease agreements .
income from the investment in these retail store leases during the years ended december 31 , 2008 , 2007 and 2006 , was approximately $ 2.7 million , $ 1.2 million and $ 1.3 million , respectively .
these amounts represent sublease revenues during the years ended december 31 , 2008 , 2007 and 2006 , of approximately $ 7.1 million , $ 7.7 million and $ 8

## Phase 6: Supervised Fine-Tuning (SFT)

This phase trains the model on Gemini-generated reasoning traces to teach it:
1. The output format (`<think>`, `<answer>` tags)
2. Step-by-step reasoning patterns


In [13]:
# Attach LoRA after base evaluation so SFT/GRPO train the adapters
if not CONFIG.get("apply_lora_before_base_eval", False):
    print("\n🔧 Attaching LoRA adapters for training...")
    model = apply_lora(model)


🔧 Attaching LoRA adapters for training...
Configuring LoRA adapters...
  Rank: 64 (higher for complex reasoning)
  Alpha: 64


Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.



✓ LoRA configured
  Trainable: 161,480,704 params (3.20%)
  Total: 5,053,027,840 params


In [14]:
from trl import SFTConfig, SFTTrainer

def prepare_sft_dataset(bootstrap_data: list) -> Dataset:
    """Prepare dataset for SFT training."""
    sft_samples = []
    
    for sample in bootstrap_data:
        # Build the full conversation
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Context:\n{sample['context'][:CONFIG.get('eval_prompt_context_chars', 6000)]}\n\nQuestion: {sample['question']}"},
            {"role": "assistant", "content": sample['reasoning_trace']}, 
        ]
        
        # Format as text using the tokenizer's chat template
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        sft_samples.append({"text": text})
    
    return Dataset.from_list(sft_samples)


if bootstrap_data:
    sft_dataset = prepare_sft_dataset(bootstrap_data)
    print(f"✓ SFT dataset prepared: {len(sft_dataset)} samples")
    
    # Show sample
    print(f"\nSample (first 500 chars):\n{sft_dataset[0]['text'][:500]}...")
else:
    sft_dataset = None
    print("⚠ No bootstrap data - SFT will be skipped")


✓ SFT dataset prepared: 499 samples

Sample (first 500 chars):
<|im_start|>system
You are an expert financial analyst. Answer financial questions accurately using step-by-step reasoning.

Follow this exact format:

1. Think through the problem step-by-step in <think> tags
2. Provide the final answer in <answer> tags

Rules:
- Do NOT write any code.
- Put the final answer ONLY inside <answer>...</answer>.

Example:
<think>
We need percent change from $100M to $150M.
Percent change = (150 - 100) / 100 = 0.5 = 50%.
</think>

<answer>50%</answer><|im_end|>
<|im...


In [15]:
SFT_OUTPUT_DIR = OUTPUT_DIR / "sft-checkpoint"

# Check if SFT should be skipped
if CONFIG["skip_sft"]:
    print("="*60)
    print("⏭️ SKIPPING SFT PHASE")
    print("="*60)
    print("  Reason: skip_sft=True in CONFIG")
    print("  The base Qwen2.5-7B-Instruct model already has good format compliance (~80%)")
    print("  Proceeding directly to GRPO with the base model + LoRA adapters")
    print("="*60)
elif sft_dataset:
    print("="*60)
    print("Starting SFT Training")
    print("="*60)
    print(f"  Samples: {len(sft_dataset)}")
    print(f"  Epochs: {CONFIG['sft_epochs']}")
    print(f"  Learning Rate: {CONFIG['sft_learning_rate']}")
    
    # Configure SFT training
    sft_config = SFTConfig(
        output_dir=str(SFT_OUTPUT_DIR),
        num_train_epochs=CONFIG["sft_epochs"],
        per_device_train_batch_size=CONFIG["sft_batch_size"],
        gradient_accumulation_steps=CONFIG["sft_gradient_accumulation"],
        learning_rate=CONFIG["sft_learning_rate"],
        logging_steps=10,
        save_steps=100,
        save_total_limit=1,
        max_seq_length=CONFIG["max_seq_length"],
        dataset_text_field="text",
        packing=False,
    )
    
    # Initialize SFT trainer
    sft_trainer = SFTTrainer(
        model=model,
        processing_class=tokenizer,
        train_dataset=sft_dataset,
        args=sft_config,
    )
    
    # Train
    sft_trainer.train()
    
    # Save SFT checkpoint
    model.save_pretrained(SFT_OUTPUT_DIR)
    tokenizer.save_pretrained(SFT_OUTPUT_DIR)
    
    print("\n" + "="*60)
    print("✓ SFT Training Complete")
    print("="*60)
    print(f"  Model saved to: {SFT_OUTPUT_DIR}")
else:
    print("⚠ Skipping SFT - no bootstrap data available")
    print("  The model will proceed directly to GRPO (may have lower performance)")


Starting SFT Training
  Samples: 499
  Epochs: 1
  Learning Rate: 2e-05


[trl.trainer.sft_trainer|WARNING]You are using a per_device_train_batch_size of 1 with padding-free training. Using a batch size of 1 anihilate the benefits of padding-free training. Please consider increasing the batch size to at least 2.


Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/499 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 499 | Num Epochs = 1 | Total steps = 125
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.667600
20,1.481800
30,1.192300
40,1.116300
50,1.020700
60,0.963000
70,0.988700
80,1.020000
90,0.917500
100,0.941500



✓ SFT Training Complete
  Model saved to: /workspace/models/checkpoints/grpo-financial-v2/sft-checkpoint


## Phase 7: Evaluate After SFT (Before GRPO)


In [16]:
# Evaluation after SFT (only if SFT was run)
# If SFT was skipped, we skip this evaluation since the base model was already evaluated

if CONFIG["skip_sft"]:
    print("="*60)
    print("⏭️ SKIPPING POST-SFT EVALUATION (SFT was not run)")
    print("="*60)
    print("  The base model was already evaluated above.")
    print("  Pre-GRPO results will show as N/A in comparison.")
    pre_grpo_results = None
    pre_grpo_accuracy = None
    pre_grpo_format_rate = None
else:
    print("📊 Evaluating model AFTER SFT (before GRPO)...")
    eval_label = "SFT Model (After SFT, Before GRPO)"


📊 Evaluating model AFTER SFT (before GRPO)...


In [17]:
# Evaluate model after SFT (only if SFT was run)
if not CONFIG["skip_sft"]:
    pre_grpo_results, pre_grpo_accuracy, pre_grpo_format_rate = evaluate_model(
        model, tokenizer, eval_df, 
        eval_label
    )



📊 Evaluating: SFT Model (After SFT, Before GRPO)

────────────────────────────────────────────────────────────────────────────────
[1/20] ID: Single_KIM/2008/page_126.pdf-2 | File: KIM/2008/page_126.pdf

🧾 CONTEXT:
PRE-TEXT:
kimco realty corporation and subsidiaries notes to consolidated financial statements , continued investment in retail store leases the company has interests in various retail store leases relating to the anchor store premises in neighborhood and community shopping centers .
these premises have been sublet to retailers who lease the stores pursuant to net lease agreements .
income from the investment in these retail store leases during the years ended december 31 , 2008 , 2007 and 2006 , was approximately $ 2.7 million , $ 1.2 million and $ 1.3 million , respectively .
these amounts represent sublease revenues during the years ended december 31 , 2008 , 2007 and 2006 , of approximately $ 7.1 million , $ 7.7 million and $ 8.2 million , respectively , less related ex

## Phase 8: GRPO Training

Now we run GRPO (Group Relative Policy Optimization) on the SFT-trained model to further improve accuracy through reinforcement learning.


In [18]:
# Define reward functions for GRPO
# NOTE: No code generation - just reasoning + final answer
# Goal: make accuracy dominate the reward signal; formatting is a small shaping term.


def _extract_predicted_answer(text: str) -> str:
    """Extract a predicted answer from a completion (lenient)."""
    if not text:
        return ""

    # Prefer <answer>...</answer>
    m = re.search(r"<answer>(.*?)(?:</answer>|$)", text, re.DOTALL | re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # Fallback: "Answer: ..."
    m = re.search(r"(?:answer|final answer|result)\s*:\s*([^\n]+)", text, re.IGNORECASE)
    if m:
        return m.group(1).strip()

    # Last resort: numeric
    nums = re.findall(r"-?\d+\.?\d*%?", text)
    return nums[-1].strip() if nums else ""


def _numeric_close(pred: str, gold: str) -> float | None:
    """Return relative error if both parse as numbers, else None."""
    try:
        pnums = re.findall(r"-?\d+\.?\d*", pred)
        gnums = re.findall(r"-?\d+\.?\d*", gold)
        if not pnums or not gnums:
            return None
        p = float(pnums[-1])
        g = float(gnums[-1])
        return abs(p - g) / (abs(g) + 1e-10)
    except Exception:
        return None


def format_reward(completions: list, **kwargs) -> list:
    """Small shaping reward for producing a clean answer format."""
    rewards = []
    for completion in completions:
        content = completion[0]["content"] if isinstance(completion, list) else str(completion)
        t = content or ""
        r = 0.0

        # Encourage answer tags (stronger than think tags)
        has_ans_open = "<answer>" in t
        has_ans_close = "</answer>" in t
        if has_ans_open:
            r += 0.05
        if has_ans_close:
            r += 0.05

        # Encourage think tags but keep small
        has_th_open = "<think>" in t
        has_th_close = "</think>" in t
        if has_th_open:
            r += 0.02
        if has_th_close:
            r += 0.02

        # Encourage non-empty extracted answer
        pred = _extract_predicted_answer(t)
        if pred:
            r += 0.06

        # Cap shaping so it doesn't dominate
        rewards.append(min(r, 0.20))

    return rewards


def accuracy_reward(completions: list, answer: list, **kwargs) -> list:
    """Main reward for answer accuracy (dominant term)."""
    rewards = []
    for completion, gold in zip(completions, answer):
        content = completion[0]["content"] if isinstance(completion, list) else str(completion)
        pred = _extract_predicted_answer(content).strip().lower()
        gold_str = str(gold).strip().lower()

        if not pred:
            rewards.append(0.0)
            continue

        # Exact
        if pred == gold_str:
            rewards.append(1.50)
            continue

        # Containment / close string
        if gold_str in pred or pred in gold_str:
            rewards.append(0.90)
            continue

        # Numeric closeness
        rel_err = _numeric_close(pred, gold_str)
        if rel_err is not None:
            if rel_err < 0.01:
                rewards.append(1.20)
            elif rel_err < 0.05:
                rewards.append(0.80)
            elif rel_err < 0.10:
                rewards.append(0.40)
            else:
                rewards.append(0.0)
        else:
            rewards.append(0.0)

    return rewards

print("✓ GRPO reward functions defined (accuracy-dominant)")

# Test (no code)
test = [[{"content": "<think>analyzing the data</think>\n<answer>25.5%</answer>"}]]
print(f"  Format reward: {format_reward(test)}")
print(f"  Accuracy reward: {accuracy_reward(test, ['25.5%'])}")


✓ GRPO reward functions defined (accuracy-dominant)
  Format reward: [0.2]
  Accuracy reward: [1.5]


In [19]:
# Configure GRPO training
from unsloth import PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

from trl import GRPOConfig, GRPOTrainer

# Prepare GRPO dataset (excludes bootstrap and evaluation samples)
grpo_dataset = Dataset.from_pandas(grpo_df[['prompt', 'answer']])
print(f"GRPO training dataset: {len(grpo_dataset)} samples")

# Training config with longer completion length for full reasoning
grpo_config = GRPOConfig(
    output_dir=str(OUTPUT_DIR / "grpo-checkpoint"),
    learning_rate=CONFIG["grpo_learning_rate"],
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=CONFIG["grpo_max_steps"],
    num_generations=CONFIG["grpo_num_generations"],
    max_completion_length=CONFIG["grpo_max_completion_length"],  # Longer for full traces
    logging_steps=10,
    use_vllm=False,
    save_steps=100,
    save_total_limit=2,
)

print(f"\nGRPO Config:")
print(f"  Max steps: {grpo_config.max_steps}")
print(f"  Max completion length: {grpo_config.max_completion_length}")
print(f"  Generations per prompt: {grpo_config.num_generations}")


Unsloth: UnslothBCOTrainer is already patched.
Unsloth: UnslothCPOTrainer is already patched.
Unsloth: UnslothDPOTrainer is already patched.
Unsloth: UnslothGKDTrainer is already patched.
Unsloth: UnslothGRPOTrainer is already patched.
Unsloth: UnslothKTOTrainer is already patched.
Unsloth: UnslothNashMDTrainer is already patched.
Unsloth: UnslothOnlineDPOTrainer is already patched.
Unsloth: UnslothORPOTrainer is already patched.
Unsloth: UnslothPPOTrainer is already patched.
Unsloth: UnslothPRMTrainer is already patched.
Unsloth: UnslothRewardTrainer is already patched.
Unsloth: UnslothRLOOTrainer is already patched.
Unsloth: UnslothSFTTrainer is already patched.
Unsloth: UnslothXPOTrainer is already patched.
GRPO training dataset: 1590 samples

GRPO Config:
  Max steps: 200
  Max completion length: 2048
  Generations per prompt: 4


In [20]:
# Initialize GRPO trainer and train
grpo_trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[format_reward, accuracy_reward],
    args=grpo_config,
    train_dataset=grpo_dataset,
)

print("="*60)
print("Starting GRPO Training")
print("="*60)
print(f"  Starting from: {'SFT model' if sft_dataset else 'Base model'}")
print(f"  Training samples: {len(grpo_dataset)}")
print(f"  Max steps: {grpo_config.max_steps}")

grpo_trainer.train()

print("\n" + "="*60)
print("✓ GRPO Training Complete")
print("="*60)


Starting GRPO Training
  Starting from: SFT model
  Training samples: 1590
  Max steps: 200


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,590 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 161,480,704 of 7,777,097,216 (2.08% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / format_reward / mean,rewards / format_reward / std,rewards / accuracy_reward / mean,rewards / accuracy_reward / std
10,0.000300,0.090000,0.020000,404.575000,233.400000,667.600000,0.000000,404.575000,233.400000,667.600000,0,0,0,0,0,0.309850,0.060000,0.000000,0.030000,0.020000
20,0.000300,0.063500,0.007000,458.625000,254.400000,712.300000,0.000000,458.625000,254.400000,712.300000,No Log,No Log,No Log,No Log,No Log,0.318108,0.063500,0.007000,0.000000,0.000000
30,0.000400,0.080000,0.040000,223.425000,107.700000,450.500000,0.000000,223.425000,107.700000,450.500000,No Log,No Log,No Log,No Log,No Log,0.393241,0.060000,0.000000,0.020000,0.040000
40,0.000300,0.060000,0.000000,383.650000,146.700000,754.000000,0.000000,383.650000,146.700000,754.000000,No Log,No Log,No Log,No Log,No Log,0.341393,0.060000,0.000000,0.000000,0.000000
50,0.000400,0.091000,0.040000,260.450000,105.100000,426.000000,0.000000,260.450000,105.100000,426.000000,No Log,No Log,No Log,No Log,No Log,0.419211,0.081000,0.020000,0.010000,0.020000
60,0.000300,0.064000,0.017464,450.700000,217.900000,770.700000,0.000000,450.700000,217.900000,770.700000,No Log,No Log,No Log,No Log,No Log,0.319724,0.064000,0.017464,0.000000,0.000000
70,0.000400,0.243000,0.052000,402.750000,223.200000,634.400000,0.025000,363.741669,223.200000,502.100000,No Log,No Log,No Log,No Log,No Log,0.425740,0.070500,0.007000,0.172500,0.045000
80,0.000300,0.089500,0.059000,402.450000,184.600000,642.800000,0.000000,402.450000,184.600000,642.800000,No Log,No Log,No Log,No Log,No Log,0.334896,0.067000,0.014000,0.022500,0.045000
90,0.000400,0.083000,0.044568,392.650000,209.800000,635.600000,0.000000,392.650000,209.800000,635.600000,No Log,No Log,No Log,No Log,No Log,0.390212,0.083000,0.044568,0.000000,0.000000
100,0.000400,0.108500,0.086201,475.750000,214.400000,787.300000,0.025000,442.625000,214.400000,667.600000,No Log,No Log,No Log,No Log,No Log,0.350452,0.086000,0.041201,0.022500,0.045000



✓ GRPO Training Complete


In [21]:
# Save final model
FINAL_MODEL_DIR = OUTPUT_DIR / "final-model"
FINAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)
model.save_pretrained(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)
print(f"✓ Final model saved to: {FINAL_MODEL_DIR}")


✓ Final model saved to: /workspace/models/checkpoints/grpo-financial-v2/final-model


## Phase 9: Final Evaluation & Comparison


In [22]:
# Evaluate final model (after SFT + GRPO)
final_results, final_accuracy, final_format_rate = evaluate_model(
    model, tokenizer, eval_df, "Final Model (After SFT + GRPO)"
)



📊 Evaluating: Final Model (After SFT + GRPO)

────────────────────────────────────────────────────────────────────────────────
[1/20] ID: Single_KIM/2008/page_126.pdf-2 | File: KIM/2008/page_126.pdf

🧾 CONTEXT:
PRE-TEXT:
kimco realty corporation and subsidiaries notes to consolidated financial statements , continued investment in retail store leases the company has interests in various retail store leases relating to the anchor store premises in neighborhood and community shopping centers .
these premises have been sublet to retailers who lease the stores pursuant to net lease agreements .
income from the investment in these retail store leases during the years ended december 31 , 2008 , 2007 and 2006 , was approximately $ 2.7 million , $ 1.2 million and $ 1.3 million , respectively .
these amounts represent sublease revenues during the years ended december 31 , 2008 , 2007 and 2006 , of approximately $ 7.1 million , $ 7.7 million and $ 8.2 million , respectively , less related expens

In [23]:
# =============================================================================
# FINAL COMPARISON: All Evaluation Stages
# =============================================================================
print("="*80)
print("📊 COMPARISON: Training Pipeline Results")
print("="*80)

# Handle case where SFT was skipped (pre_grpo_* are None)
sft_was_run = pre_grpo_results is not None

print(f"\n{'Stage':<45} {'Accuracy':<15} {'Format Rate':<15}")
print("-"*80)
print(f"{'1. BASE Model (before any training)':<45} {base_accuracy:>10.1f}% {base_format_rate:>12.1f}%")

if sft_was_run:
    print(f"{'2. After SFT (before GRPO)':<45} {pre_grpo_accuracy:>10.1f}% {pre_grpo_format_rate:>12.1f}%")
else:
    print(f"{'2. After SFT (before GRPO)':<45} {'N/A':>10} {'N/A':>12}   (SFT skipped)")

print(f"{'3. After GRPO (Final)':<45} {final_accuracy:>10.1f}% {final_format_rate:>12.1f}%")
print("-"*80)

# Show improvements
print(f"\n📈 IMPROVEMENTS:")
if sft_was_run:
    print(f"  SFT Effect (Base → SFT):   {pre_grpo_accuracy - base_accuracy:>+6.1f}% accuracy, {pre_grpo_format_rate - base_format_rate:>+6.1f}% format")
    print(f"  GRPO Effect (SFT → Final): {final_accuracy - pre_grpo_accuracy:>+6.1f}% accuracy, {final_format_rate - pre_grpo_format_rate:>+6.1f}% format")
else:
    print(f"  SFT Effect:                  N/A (SFT was skipped)")
    print(f"  GRPO Effect (Base → Final): {final_accuracy - base_accuracy:>+6.1f}% accuracy, {final_format_rate - base_format_rate:>+6.1f}% format")
print(f"  Total (Base → Final):      {final_accuracy - base_accuracy:>+6.1f}% accuracy, {final_format_rate - base_format_rate:>+6.1f}% format")

# Per-question comparison (Base vs Final only if SFT was skipped)
print(f"\n{'─'*80}")
if sft_was_run:
    print("Per-Question Comparison (Base → SFT → Final):")
else:
    print("Per-Question Comparison (Base → Final):")
print(f"{'─'*80}")

comparison_data = []
for i, (base, final) in enumerate(zip(base_results, final_results)):
    base_status = "✓" if base['correct'] else "✗"
    final_status = "✓" if final['correct'] else "✗"
    
    # Get SFT result if available
    if sft_was_run:
        sft_result = pre_grpo_results[i]
        sft_status = "✓" if sft_result['correct'] else "✗"
        sft_pred = sft_result['predicted']
        sft_correct = sft_result['correct']
        sft_format_ok = sft_result['format_ok']
    else:
        sft_status = "-"
        sft_pred = "N/A"
        sft_correct = None
        sft_format_ok = None
    
    # Track overall progression (Base → Final)
    if not base['correct'] and final['correct']:
        overall = "📈 Fixed"
    elif base['correct'] and not final['correct']:
        overall = "📉 Broke"
    elif base['correct'] and final['correct']:
        overall = "✓ Always OK"
    else:
        overall = "✗ Still wrong"
    
    if sft_was_run:
        print(f"Q{i+1:2d}: Base {base_status} → SFT {sft_status} → Final {final_status} | {overall}")
    else:
        print(f"Q{i+1:2d}: Base {base_status} → Final {final_status} | {overall}")
    
    comparison_data.append({
        "question": base['question'],
        "ground_truth": base['ground_truth'],
        "base_pred": base['predicted'],
        "sft_pred": sft_pred,
        "final_pred": final['predicted'],
        "base_correct": base['correct'],
        "sft_correct": sft_correct,
        "final_correct": final['correct'],
        "base_format_ok": base['format_ok'],
        "sft_format_ok": sft_format_ok,
        "final_format_ok": final['format_ok'],
        "overall_status": overall,
    })

# Summary
fixed = sum(1 for c in comparison_data if "Fixed" in c['overall_status'])
broke = sum(1 for c in comparison_data if "Broke" in c['overall_status'])
always_ok = sum(1 for c in comparison_data if "Always OK" in c['overall_status'])
still_wrong = sum(1 for c in comparison_data if "Still wrong" in c['overall_status'])

print(f"\n{'─'*80}")
print("Overall Summary (Base → Final):")
print(f"  📈 Fixed by GRPO:      {fixed}")
print(f"  📉 Broken by GRPO:     {broke}")
print(f"  ✓ Always correct:      {always_ok}")
print(f"  ✗ Still wrong:         {still_wrong}")


📊 COMPARISON: Training Pipeline Results

Stage                                         Accuracy        Format Rate    
--------------------------------------------------------------------------------
1. BASE Model (before any training)                 45.0%        100.0%
2. After SFT (before GRPO)                          40.0%        100.0%
3. After GRPO (Final)                               30.0%        100.0%
--------------------------------------------------------------------------------

📈 IMPROVEMENTS:
  SFT Effect (Base → SFT):     -5.0% accuracy,   +0.0% format
  GRPO Effect (SFT → Final):  -10.0% accuracy,   +0.0% format
  Total (Base → Final):       -15.0% accuracy,   +0.0% format

────────────────────────────────────────────────────────────────────────────────
Per-Question Comparison (Base → SFT → Final):
────────────────────────────────────────────────────────────────────────────────
Q 1: Base ✓ → SFT ✗ → Final ✗ | 📉 Broke
Q 2: Base ✗ → SFT ✗ → Final ✗ | ✗ Still wrong
Q 3: 

In [24]:
# Save comparison results to CSV
comparison_df = pd.DataFrame(comparison_data)
output_csv = OUTPUT_DIR / "comparison_results.csv"
comparison_df.to_csv(output_csv, index=False)
print(f"✓ Results saved to: {output_csv}")

# Display summary table
print("\n" + "="*70)
print("Results Table")
print("="*70)
if sft_was_run:
    display(comparison_df[["ground_truth", "base_pred", "sft_pred", "final_pred", "overall_status"]])
else:
    display(comparison_df[["ground_truth", "base_pred", "final_pred", "overall_status"]])


✓ Results saved to: /workspace/models/checkpoints/grpo-financial-v2/comparison_results.csv

Results Table


,ground_truth,base_pred,sft_pred,final_pred,overall_status
0,-7.8%,-7.79%,125%,-0.1%,📉 Broke
1,4.6%,4.9%,5.1%,1%,✗ Still wrong
2,273%,222.92%,223.4%,272.6%,📈 Fixed
3,26.8%,Insufficient data to determine the growth rate.,6.67%,18.5%,✗ Still wrong
4,-489.2,731.7,1110.8,-1281.3,✗ Still wrong
5,41661517,"41,661,517",57261177,55224783,✗ Still wrong
6,14.3%,14%,10%,15.1%,📉 Broke
7,-23.8%,4268,$22223,$4268,✗ Still wrong
8,13.25%,13.2%,13.3%,29.3%,📉 Broke
9,30%,110%,29%,118%,✗ Still wrong


## Done!

The training pipeline is complete. The model went through:

1. **Bootstrap Data Generation**: Gemini created high-quality reasoning traces (optional)
2. **SFT Phase**: Taught the model the output format and reasoning patterns (skipped by default if base model already has good format compliance)
3. **GRPO Phase**: Reinforcement learning to improve accuracy

**Key files:**
- Final model: `models/checkpoints/grpo-financial-v2/final-model/`
- SFT checkpoint: `models/checkpoints/grpo-financial-v2/sft-checkpoint/` (if SFT was run)
- Bootstrap data: `data/processed/bootstrap_data.json`
- Comparison results: `models/checkpoints/grpo-financial-v2/comparison_results.csv`

**Notes:**
- Set `CONFIG["skip_sft"] = False` if you want to run SFT before GRPO
- The base Qwen2.5-7B-Instruct model already achieves ~80% format compliance, so SFT may not be necessary
